In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np

# Machine learning libraries
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Technical indicator library
import talib as ta

# Data import library
import yfinance as yf

#Data visualisation
import plotly.io as pio
pio.renderers.default = 'iframe'# 'iframe' # or 'notebook' or 'colab' or 'jupyterlab'
import plotly.graph_objs as go

### Loading Data from yelp

In [8]:
df = yf.download('TSLA', start="2022-03-28", end="2022-03-29",period = '1d', interval = '1m')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-03-28 09:30:00-04:00,1065.099976,1068.500000,1057.500000,1063.989990,1063.989990,3289164
2022-03-28 09:31:00-04:00,1064.040039,1070.000000,1059.890015,1062.848999,1062.848999,283958
2022-03-28 09:32:00-04:00,1063.000000,1063.000000,1056.750000,1060.219971,1060.219971,237168
2022-03-28 09:33:00-04:00,1060.454956,1061.839966,1053.599976,1058.989990,1058.989990,276402
2022-03-28 09:34:00-04:00,1056.750000,1057.989990,1055.000000,1055.633301,1055.633301,180339
...,...,...,...,...,...,...
2022-03-28 15:56:00-04:00,1086.724976,1087.270020,1086.010010,1087.010132,1087.010132,86618
2022-03-28 15:57:00-04:00,1087.135010,1088.439941,1086.949951,1088.275024,1088.275024,115840
2022-03-28 15:58:00-04:00,1088.290039,1090.200439,1088.239990,1089.030151,1089.030151,187718


In [9]:
fig = go.Figure()
fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'], name = 'market data'))
fig.update_layout(title='Tesla price',yaxis_title='Stock Price (USD per Shares)')
# X-Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="30m", step="minute", stepmode="backward"),
            dict(count=6, label="90m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(step="all")])))
fig.show()

### Data Cleaning

In [10]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,391.000000,391.000000,391.000000,391.000000,391.000000,3.910000e+02
mean,1084.167750,1085.238759,1082.994292,1084.167533,1084.167533,8.481271e+04
std,9.228444,8.949111,9.637091,9.299391,9.299391,1.726722e+05
min,1056.000000,1057.989990,1053.599976,1055.633301,1055.633301,0.000000e+00
25%,1078.614929,1079.719910,1077.820068,1078.714966,1078.714966,3.806300e+04
50%,1087.400024,1088.430054,1086.443237,1087.425049,1087.425049,5.886900e+04
75%,1091.202454,1091.994934,1090.265015,1091.194946,1091.194946,9.519600e+04
max,1099.569946,1099.569946,1099.569946,1099.569946,1099.569946,3.289164e+06


In [11]:
# Drop the rows with zero volume traded
df = df.drop(df[df['Volume'] == 0].index)

### RSI (Relative Strength Index) is a well-known trading indicator based on momentum strategy.

In [12]:
# Create a variable n with a value of 10
n = 10
# Create a column by name, RSI and assign the calculation of RSI to it
df['RSI'] = ta.RSI(np.array(df['Close'].shift(1)), timeperiod=n)
df

,Open,High,Low,Close,Adj Close,Volume,RSI
Datetime,,,,,,,
2022-03-28 09:30:00-04:00,1065.099976,1068.500000,1057.500000,1063.989990,1063.989990,3289164,NaN
2022-03-28 09:31:00-04:00,1064.040039,1070.000000,1059.890015,1062.848999,1062.848999,283958,NaN
2022-03-28 09:32:00-04:00,1063.000000,1063.000000,1056.750000,1060.219971,1060.219971,237168,NaN
2022-03-28 09:33:00-04:00,1060.454956,1061.839966,1053.599976,1058.989990,1058.989990,276402,NaN
2022-03-28 09:34:00-04:00,1056.750000,1057.989990,1055.000000,1055.633301,1055.633301,180339,NaN
...,...,...,...,...,...,...,...
2022-03-28 15:55:00-04:00,1086.699951,1087.000000,1085.520020,1086.710083,1086.710083,98977,41.672071
2022-03-28 15:56:00-04:00,1086.724976,1087.270020,1086.010010,1087.010132,1087.010132,86618,42.519163
2022-03-28 15:57:00-04:00,1087.135010,1088.439941,1086.949951,1088.275024,1088.275024,115840,44.184949


In [14]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Line(x=df.index, y=df.Open, name = 'Price'), row =1, col=1)
fig.add_trace(go.Line(x=df.index, y=df.RSI, name = 'RSI'), row =2, col=1)
fig.update_layout(title='Relative Strength Index and Stock Price for TESLA', yaxis_title='Stock Price (USD per Shares)')
fig.show()